#**Sentence classification** (11 points)

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 28.4 MB/s 
     |████████████████████████████████| 115 kB 68.0 MB/s 
     |████████████████████████████████| 182 kB 72.4 MB/s 
     |████████████████████████████████| 212 kB 74.5 MB/s 
     |████████████████████████████████| 127 kB 68.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import datasets

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 36.1 MB/s 
     |████████████████████████████████| 7.6 MB 52.8 MB/s 


#Question 1

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("imdb")
raw_datasets
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Processing the data

In [ ]:
def tokenize_function(df):
  """
    Tokenize a sequence of text
    Args:
      df: the input data
  """
  return tokenizer(df["text"], truncation=True)

In [ ]:
#Apply the tokenization function on all datasets
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

#We use the DataCollatorWithPadding function from the Transformers library to
#collate the samples inside a batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

Training

In [ ]:
#Define the Trainer

from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer", num_train_epochs = 1)

In [ ]:
#Define the model

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

In [ ]:
#Define a Trainer with all the objects constructed so far

from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3125
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster th

Step,Training Loss
500,0.384600
1000,0.334100
1500,0.308300
2000,0.286100
2500,0.261900
3000,0.264500


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1500
Configuration saved in test-trainer/checkpoint-1500/config.json
Model weights saved in test-trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved

TrainOutput(global_step=3125, training_loss=0.306139267578125, metrics={'train_runtime': 1145.415, 'train_samples_per_second': 21.826, 'train_steps_per_second': 2.728, 'total_flos': 3121546643857248.0, 'train_loss': 0.306139267578125, 'epoch': 1.0})

Evaluation

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 25000
  Batch size = 8


(25000, 2) (25000,)


In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

#Question 2 (2 points)
##Evaluate the model in term of accuracy on the test data

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.3 MB/s 


The evaluation of the model accuracy:

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.9292}

#Question 3 (2 points)
##For at least 2 samples which have been wrongly classified in the test set, try explaining why the model could have been wrong.

In [ ]:
train_data, test_data = load_dataset('imdb', split =['train', 'test'])
df_pandas_test = test_data.to_pandas()
df_pandas_test

  0%|          | 0/2 [00:00<?, ?it/s]

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0
...,...,...
24995,Just got around to seeing Monster Man yesterda...,1
24996,I got this as part of a competition prize. I w...,1
24997,I got Monster Man in a box set of three films ...,1
24998,"Five minutes in, i started to feel how naff th...",1


In [ ]:
df_pandas_test['predictions'] = preds

In [ ]:
df_pandas_test

,text,label,predictions
0,I love sci-fi and am willing to put up with a ...,0,0
1,"Worth the entertainment value of a rental, esp...",0,0
2,its a totally average film with a few semi-alr...,0,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0,0
4,"First off let me say, If you haven't enjoyed a...",0,1
...,...,...,...
24995,Just got around to seeing Monster Man yesterda...,1,1
24996,I got this as part of a competition prize. I w...,1,0
24997,I got Monster Man in a box set of three films ...,1,1
24998,"Five minutes in, i started to feel how naff th...",1,1


In [ ]:
#Check where the classification failed

import numpy as np

print(np.where(df_pandas_test.predictions != df_pandas_test.label)[0])

[    4    18    20 ... 24981 24993 24996]


We will take the example of the line 4 and 18 where the classification failed

In [ ]:
df_pandas_test.iloc[4]

text           First off let me say, If you haven't enjoyed a...
label                                                          0
predictions                                                    1
Name: 4, dtype: object

In [ ]:
# To see the whole text

df_pandas_test.iloc[4]['text']

"First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie. Most of these movies may not have the best plots or best actors but I enjoy these kinds of movies for what they are. This movie is much better than any of the movies the other action guys (Segal and Dolph) have thought about putting out the past few years. Van Damme is good in the movie, the movie is only worth watching to Van Damme fans. It is not as good as Wake of Death (which i highly recommend to anyone of likes Van Damme) or In hell but, in my opinion it's worth watching. It has the same type of feel to it as Nowhere to Run. Good fun stuff!"

In [ ]:
df_pandas_test.iloc[18]

text           Ben, (Rupert Grint), is a deeply unhappy adole...
label                                                          0
predictions                                                    1
Name: 18, dtype: object

In [ ]:
df_pandas_test.iloc[18]['text']

"Ben, (Rupert Grint), is a deeply unhappy adolescent, the son of his unhappily married parents. His father, (Nicholas Farrell), is a vicar and his mother, (Laura Linney), is ... well, let's just say she's a somewhat hypocritical soldier in Jesus' army. It's only when he takes a summer job as an assistant to a foul-mouthed, eccentric, once-famous and now-forgotten actress Evie Walton, (Julie Walters), that he finally finds himself in true 'Harold and Maude' fashion. Of course, Evie is deeply unhappy herself and it's only when these two sad sacks find each other that they can put their mutual misery aside and hit the road to happiness.<br /><br />Of course it's corny and sentimental and very predictable but it has a hard side to it, too and Walters, who could sleep-walk her way through this sort of thing if she wanted, is excellent. It's when she puts the craziness to one side and finds the pathos in the character, (like hitting the bottle and throwing up in the sink), that she's at her 

##Explanation
We think the reason why the model could have been wrong might be:


*   We only fine-tuned the model for one epoch
*   The input texts were not cleaned, there were no pre-treaments
*   It can be hard for the model to differentiate because the text can be composed with positive and negative sentiments. The text doesn't show a strong positive or negative feeling as shown on the example 18. 
*   The example 4 contains a lof of positive tokens like "enjoy", "good","best" etc... which can trick the model. Moreover, the model has a hard time comprehending the token "not" as a negation when placed next to positive token.



#Question 4 (2 points)
##What are the advantages and inconvenient of using this model in production compared to the naive Bayes we implemented in the first part of the course?

##Advantages:

*   Distilbert is faster and provides the best results in terms of accuracy since this model has been pre-trained and fine-tuned for text classification
*   Distilbert has a better contextual understanding, it can tell the differences between words with the same spelling based on the context. On the other hand, Naive Bayes model assumes all features are independent.

##Disadvantages:

*   Naive Bayes is easier to understand and implement
*   Naive Bayes does not require fine-tuning
*   Naive Bayes can take a lot of time to train since we don't have a pre-trained model. 



